In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

# replicate figure 7 from carlesten 

setup: calculate the proportion of transaction fees earned by an adversary playing the pure selfish strategy given block times modelled as an exponential distribution.

In [19]:
np.random.exponential()

1.6075376662009329

In [216]:
state = 0
alpha = 0.3

honestRews = 0
attackRews = 0
remainRews = 0
honestBlcs = 0
attackBlcs = 0

for _ in tqdm(range(int(1e5))):
    blockTime = np.random.exponential(1/100000)
    attBlock  = np.random.binomial(1, alpha)

    if state == 0 and attBlock:
        state       = 1
        remainRews += blockTime  # unclaimed exec rewards on private branch
    elif state == 0 and not attBlock:
        honestRews += blockTime
        honestBlcs += 1

    if state == 1 and attBlock:
        state       = 2
        attackRews += (remainRews + blockTime)  # attacker locks in both blocks worth of exec rewards
        attackBlcs += 2
        remainRews  = 0
    elif state == 1 and not attBlock:
        nextBlockTime = np.random.exponential()
        nextAttBlock  = np.random.binomial(1, alpha)
        # attacker wins all three time period's worth of exec rewards
        if nextAttBlock:
            attackRews += (remainRews + blockTime + nextBlockTime)
            attackBlcs += 2
        #   honest wins all three time period's worth of exec rewards
        else:
            honestRews += (remainRews + blockTime + nextBlockTime)
            honestBlcs += 2
        remainRews = 0

    if state == 2 and attBlock:
        state        = 3
        attackRews  += (remainRews + blockTime)
        remainRews   = 0
        attackBlcs  += 1
    elif state == 2 and not attBlock:
        nextBlockTime = np.random.exponential()
        nextAttBlock  = np.random.binomial(1, alpha)
        # attacker wins all outstanding exec rewards
        if nextAttBlock:
            attackRews += (remainRews + blockTime + nextBlockTime)
            attackBlcs  += 1
        #   honest wins all outstanding exec rewards
        else:
            honestRews  += (remainRews + blockTime + nextBlockTime)
            honestBlcs  += 1
        remainRews = 0

    if state > 2 and attBlock:
        state        += 1
        attackRews   += (remainRews + blockTime)
        remainRews    = 0
        attackBlcs   += 1
    elif state > 2 and not attBlock:
        state      -= 1
        remainRews += blockTime

totalRews = honestRews + attackRews
totalBlcs = honestBlcs + attackBlcs
(honestRews/totalRews, attackRews/totalRews), (honestBlcs/totalBlcs, attackBlcs/totalBlcs), honestBlcs, totalBlcs

100%|██████████| 100000/100000 [00:00<00:00, 125992.68it/s]


((0.6974516120924992, 0.3025483879075008),
 (0.27788669309254305, 0.722113306907457),
 19419,
 69881)

In [207]:
int(1e5)

100000